## Coletando dados - Viva Real

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/'

In [3]:
c = requests.get(url).content
soup = BeautifulSoup(c)

In [4]:
aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}) 
len(aluguel_tag)

0

#### Definindo coleta das páginas da URL:

In [5]:
paginas = ['']
for i in range(2,5):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)

?pagina=2
?pagina=3
?pagina=4


In [6]:
paginas

['', '?pagina=2', '?pagina=3', '?pagina=4']

#### Criando DF para visualização dos dados.

In [7]:
import pandas as pd
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista}
    df = pd.DataFrame(data)
    return df

In [8]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    
    aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}) 
    aluguel_lista = [p.text for p in aluguel_tag] 
    print(len(aluguel_lista)) 
    
    quartos_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'}) 
    quartos_lista = [en.text for en in quartos_tag] 
    print(len(quartos_lista)) 
    
    endereco_tag = soup.find_all('span', {'class': 'property-card__address'}) 
    endereco_lista = [fg.text for fg in endereco_tag] 
    print(len(endereco_lista)) 
    
    banheiro_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}) 
    banheiro_lista = [yp.text for yp in banheiro_tag] 
    print(len(banheiro_lista))
    
    ### Vamos ter criar uma função para transformar listas em df
    df=converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista)
    
    return df

In [9]:
paginas

['', '?pagina=2', '?pagina=3', '?pagina=4']

In [10]:
URL = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/{}'

In [11]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)

df_final = pd.concat(df_lista)

Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/
36
36
36
36
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=2
36
36
36
36
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=3
36
36
36
36
Coletando URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=4
36
36
36
36


#### Visualizando o DF.

In [12]:
df_final.head()

,aluguel,quartos,endereco,banheiro
0,R$ 6.000 /Mês,2 Quartos,"Rua Haddock Lobo, 547 - Cerqueira César, São P...",3 Banheiros
1,R$ 3.500 /Mês,2 Quartos,"Avenida Lins de Vasconcelos, 2805 - Vila Maria...",2 Banheiros
2,R$ 3.300 /Mês,1 Quarto,"Rua Constantino de Sousa, 1144 - Campo Belo, S...",1 Banheiro
3,R$ 4.600 /Mês,2 Quartos,"Rua Cunha Gago, 181 - Pinheiros, São Paulo - SP",2 Banheiros
4,R$ 2.900 /Mês,3 Quartos,"Rua Pensilvânia, 240 - Brooklin, São Paulo - SP",3 Banheiros


In [13]:
df_final['bairro'] = df_final.endereco.str.split('\s-').str[1]
df_final['rua'] = df_final.endereco.str.split('\s-').str[0]
df_final['bairro'] = df_final.bairro.str.split(',').str[0]

df_final.head()

,aluguel,quartos,endereco,banheiro,bairro,rua
0,R$ 6.000 /Mês,2 Quartos,"Rua Haddock Lobo, 547 - Cerqueira César, São P...",3 Banheiros,Cerqueira César,"Rua Haddock Lobo, 547"
1,R$ 3.500 /Mês,2 Quartos,"Avenida Lins de Vasconcelos, 2805 - Vila Maria...",2 Banheiros,Vila Mariana,"Avenida Lins de Vasconcelos, 2805"
2,R$ 3.300 /Mês,1 Quarto,"Rua Constantino de Sousa, 1144 - Campo Belo, S...",1 Banheiro,Campo Belo,"Rua Constantino de Sousa, 1144"
3,R$ 4.600 /Mês,2 Quartos,"Rua Cunha Gago, 181 - Pinheiros, São Paulo - SP",2 Banheiros,Pinheiros,"Rua Cunha Gago, 181"
4,R$ 2.900 /Mês,3 Quartos,"Rua Pensilvânia, 240 - Brooklin, São Paulo - SP",3 Banheiros,Brooklin,"Rua Pensilvânia, 240"
